# Let's Define our Goal for Predictions

Let's use the data we pulled, and determine the target outputs for our model to predict correctly!

In [8]:
# Set up the env.
#!conda init
#!conda env list ## to see the availble options
#!conda activate civil_unrest ## to activate the targeted env

In [9]:
# Import the necessary module
from google.cloud import bigquery
from matplotlib import pyplot as plt
import pandas as pd
import folium
from matplotlib import colors
from geopy.distance import geodesic


## Obtaining Events Along Commute

We want only the events that exist along the commute to work

In [10]:
def expand_rectangle_by_mile(start_lat, start_lon, end_lat, end_lon):
    """
    Expands a rectangle by 3 miles in both latitude and longitude directions.
    Each coordinate is shifted outward by 1.5 miles.

    Parameters:
        start_lat (float): Starting latitude of the rectangle.
        start_lon (float): Starting longitude of the rectangle.
        end_lat (float): Ending latitude of the rectangle.
        end_lon (float): Ending longitude of the rectangle.

    Returns:
        pd.DataFrame: DataFrame containing the expanded rectangle coordinates.
    """
    # Calculate shifts of 3 miles in latitude and longitude
    threemile_in_lat = geodesic(miles=3).destination((start_lat, start_lon), 0).latitude - start_lat
    threemile_in_lon = geodesic(miles=3).destination((start_lat, start_lon), 90).longitude - start_lon

    # Determine the minimal and maximal coordinates
    min_lat = min(start_lat, end_lat)
    max_lat = max(start_lat, end_lat)
    min_lon = min(start_lon, end_lon)
    max_lon = max(start_lon, end_lon)

    # Expand the rectangle
    expanded_start_lat = min_lat - threemile_in_lat
    expanded_start_lon = min_lon - threemile_in_lon
    expanded_end_lat = max_lat + threemile_in_lat
    expanded_end_lon = max_lon + threemile_in_lon

    expanded_coords = {
        "start_lat": [expanded_start_lat],
        "start_lon": [expanded_start_lon],
        "end_lat": [expanded_end_lat],
        "end_lon": [expanded_end_lon]
    }

    return pd.DataFrame(expanded_coords)

---

Let's simulate this with my route from home to school.

Let's init our vars using my experience in Chicago. We'll later productize this entire analysis for anywhere in the world.

---

In [11]:
home_lat, home_lon = 41.91118832433419, -87.67514378155508  # nearby my home
school_lat, school_lon = 41.87300017458362, -87.62765043486581  # nearby my school

expanded_coords = expand_rectangle_by_mile(home_lat, home_lon, school_lat, school_lon)

# Create a DataFrame with home, school, and expanded coordinates
coords = pd.DataFrame({
    'lat': [home_lat, school_lat, expanded_coords['start_lat'].iloc[0], expanded_coords['end_lat'].iloc[0]],
    'lon': [home_lon, school_lon, expanded_coords['start_lon'].iloc[0], expanded_coords['end_lon'].iloc[0]],
    'event': ['home', 'school', 'expanded_start', 'expanded_end']
})

coords

,lat,lon,event
0,41.911188,-87.675144,home
1,41.873000,-87.627650,school
2,41.829533,-87.733337,expanded_start
3,41.954656,-87.569457,expanded_end


---

Let's visualize the region I've decided to focus on. You'll see my home and my school below.

---

In [12]:
# Create a map centered around the midpoint of the expanded rectangle
mid_lat = (expanded_coords['start_lat'] + expanded_coords['end_lat']) / 2
mid_lon = (expanded_coords['start_lon'] + expanded_coords['end_lon']) / 2
m = folium.Map(location=[mid_lat.iloc[0], mid_lon.iloc[0]], zoom_start=13)

# Add a rectangle to the map
folium.Rectangle(
    bounds=[
        [expanded_coords['start_lat'].iloc[0], expanded_coords['start_lon'].iloc[0]],
        [expanded_coords['end_lat'].iloc[0], expanded_coords['end_lon'].iloc[0]]
    ],
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2
).add_to(m)

# Add starting point marker
folium.Marker(
    location=[home_lat, home_lon],
    popup='Starting Point',
    icon=folium.Icon(color='green')
).add_to(m)

# Add ending point marker
folium.Marker(
    location=[school_lat, school_lon],
    popup='Ending Point',
    icon=folium.Icon(color='red')
).add_to(m)

# Display the map
m

---

Saving the geographic bounds to the data!

Let's ensure we can reference this elsewhere.

---

In [13]:
coords.to_csv('csv/target_location.csv', index=False)